## RSA - The Project

Name: Alijah O'Connor 

Moodle Email: alijah.oconnor@colorado.edu

Did you name your file "cuidentikey".ipynb ? Answer yes or no <br>
Yes, I did!

# Introduction

This notebook allows a user to interact with a real RSA encrypter and decrypter.  The notebook contains many functions that build up the final functionality of the RSA encrypter/decrypter.  Any person with the base understand of RSA will have no problem using the notebook.  As you will see, the functionality of RSA has been broken down into 3 different parts for the user to surf through -- 1) Generating keys (you can select your p and q values or be pseudo-randomly assigned them), 2) Encrypting a message using a public key, and 3) Decrypting an encrypted cipher (must know the d value).

If you are still curious about RSA, I explore some aspects of the schema below the aforementioned implementer.  I include a section for decrypting messages with a d value, discuss the challenges of doing this, and answer some questions that I once had about the schema before actually learning it formally.  Have fun -- I sure did!

# Table of Contents

1. <a href='#project_dependencies'>Project Dependencies</a>
2. <a href='#second_tool_set'>Second Tool Set</a>
3. <a href='#first_tool_set'>First Tool Set</a>
4. <a href='#third_tool_set'>Third Tool Set</a>
5. <a href='#putting'>Putting Everything Together</a>
6. <a href='#my_functions'>My Functions</a>
    <br>6.1. <a href='#prime_related'>Prime-Related Functions</a>
    <br>6.2. <a href='#key_related'>Key-Related Functions</a>
    <br>6.3. <a href='#user_functionality'>User Functionality</a>
7. <a href='#user_interface'>User Interface</a>
8. <a href='#section_4'>Section "4" Demo/Explanation</a>
9. <a href='#section_5'>Section "5" Narrative</a>
10. <a href='#section_6'>Section "6" Factoring and Code Breaking</a>
    <br>10.1. <a href='#naive'>Naive, Brute Force Factoring</a>
    <br>10.2. <a href='#pollards'>Advanced Factoring with Pollard's Rho</a>
    <br>10.3. <a href='#code_cracking'>Cracking Codes</a>
11. <a href='#section_7'>Section "7" Code Breaking Description</a>
12. <a href='#section_8'>Section "8" Code Breaking Successes</a>
13. <a href='#section_9'>Section "9" Theorem 2 Importance</a>
14. <a href='#section_10'>Section "10" Conclusion/Answering Questions</a>


<section id='project_dependencies'>
    
    
#### 1. Project Dependencies

In [1]:
import math, random, time

<section id='second_tool_set'>
#### 2. Second tool set

Even though this is the __second__ toolset, there are functions that you'll need to pre-process the messages before the messages are encoded and decoded by the RSA algorithm. Thats the reason we will be defining them first.

In [2]:
def Convert_Text(message):
    """
    Converts a message into a list of ascii characters corresponding to each character in the message.
    
    Args:
        message (string) - message to be converted
        
    Returns:
        (list) - contains the ascii-converted characters
    
    """
    ascii_list = []
    for char in message:
        ascii_list.append(ord(char))
    return ascii_list

In [3]:
def Convert_Num(ascii_list):
    """
    Converts an list containing ascii numbers to their corresponding unicode letters.
    
    Args:
        ascii_list - list containing ascii numbers
        
    Returns: 
        (string) - converted string from the ascii list
    """
    res_string = ''
    ascii_list_copy = ascii_list[:]
    for num in ascii_list_copy:
        res_string += chr(num)
    return res_string

In [4]:
def Convert_Binary_String(a_number):
    """
    Converts an integer into binary digits as a string. Credit to Reagan Karnes
    and Beth Spade for the implementation this algorithm -- I am using for
    the added performance boast of the built-in bin() function.
    
    Args:
        a_number (int) - the base-10 representation of a number to be converted to binary
        
    Returns:
        (string) - string form of the binary representation of a number
    """
    return bin(a_number)[2:] # start at index 2 to remove the binary-ID

<section id='first_tool_set'>
#### 3. First tool set

In [5]:
def FME(b, n, m):
    """
    Simulates Fast Modular Exponentation of b^n mod m
    
    Args:
        b (int) - Base of the exponent 
        n (str) - Binary Expansion of the Exponent
        m (int) - The (positive, non-zero) integer which divides b^n
    
    Returns:
        (int) - The result of b^n mod m
    """
    assert m > 0
    
    x = 1 # Initialize the remainder
    power = b % m
    k = len(n) # Number of bits in the exponent bit string
    
    for i in range(k-1, -1, -1): # Runs from the least to most significant digit
        if n[i] == '1':
            x = (x * power) % m
        power = (power ** 2) % m
    return x

In [6]:
def div_alg(a, d):
    """
    Simulates the division of two numbers, calculating both the quotient and the remainder of the division operation
    
    Args:
        a (int) - number to be divided by d
        d (int) - (non-zero, positive) number which divides a
    
    Returns:
        (tuple) - contains the quotient of the division and the remainder
    """
    assert (d > 0) # b must be a non-zero positive number
    q = 0 # initialize the quotient
    r = abs(a) # initialize the remainder
    
    # if a is positive
    while r >= d:
        r -= d
        q += 1
    
    # if a is negative, then the operation requires an addition substraction
    if a < 0 and r > 0:
        r = d - r
        q = -(q + 1)
    return (q, r) 

In [7]:
def Euclidean_Alg(a, b):
    """
    Simulates the Euclidean Algorithm, finding the greatest common demoninator 
    two positive numbers in a efficient manner
    
    Args:
        a (int) - First Number
        b (int) - Second Number
    
    Returns:
        (int) - greatest common denominator of a and b
    """
    while b != 0:
        r = a % b
        a = b
        b = r
    return a # The final non-zero remainder in the process, aka the gcd(a,b)
    
def extended_euclidean_alg(a,b):
    """
    Simulates the Extended Euclidean Algorithm, finding both the greatest common demoninator 
    of two positive numbers and (if an inveres exists; that is, if the two numbers are relatively prime {gcd = 1})
    the inverse of one number* (mod the other*).
    
    * The 'other number' always corresponds to the larger number that is inputted, 'one number' always corresponds
      to the smaller of the two numbers.
    
    Args:
        a (int) - First Number
        b (int) - Second Number
    
    Returns:
        (tuple) - first index:  greatest common denominator of a and b, second index:  The inverse (if one exists)
    """
    # the larger number always comes first for this algorithm design,
    #    if this is not the case, then switch a and b

    if a < b: 
        a, b = b, a 
        
    a0 = a # Intial a records the original modulo amount
       
    # if b divides a, then the gcd is equal to b and there is no inverse
    if a % b == 0:
        return b, "No Inverse"
    
    bezo_coeff_list = []
    
    i = 0
    while True:
        k = a % b
        q = a // b
        a, b = b, k
        
        # The gcd has been reached
        if k == 0:
            break
         
        # Builds a list of Bezout Coefficients to use recursively as the correct 
        # Bezout coefficient for the inverse is generated through iteration
        if i == 0:
            bezo_coeff_list.append(-1 * q)
        elif i == 1:
            bezo_coeff_list.append(1 - q * bezo_coeff_list[0])
        else:
            bezo_coeff_list.append(bezo_coeff_list[i - 2] - q * bezo_coeff_list[i - 1])
            
        i += 1
    
    # Because the mod inverse isn't unique, if negative, add the modulo amount
    mod_inverse = bezo_coeff_list.pop()
    if mod_inverse < 0:
        mod_inverse += a0
    
    # if the gcd(a,b) is not equal to 1, return the calculated gcd and report that there is no inverse 
    if a != 1:
        return a, "No Inverse"
    return a, mod_inverse


<section id='third_tool_set'>
#### 4. Third tool set

In [8]:
def Find_Public_Key_e(p, q):
    """
    Find the smallest value of e (to be used in the public key of RSA)
    that is relatively prime to (p-1)(q-1).  Function raises an error if a valid e cannot be found.
    
    Args:
        p (int) - the first of two prime numbers for the calculation
        q (int) - the second prime number for the calculation
        
    Returns:
        (int) - the e value to be used in an RSA public key
    """
    pm1 = p - 1
    qm1 = q - 1
    p1q1 = pm1 * qm1
    
    Public_Key_e = 3 # skip 1 because it isn't RP to anything, skip 2
                     # because p-1 and q-1 will both be even
    while Public_Key_e < pm1 and Public_Key_e < qm1:
        if (math.gcd(Public_Key_e, p1q1) == 1):
            return Public_Key_e
        Public_Key_e += 2 # Skip all even numbers
        
    # If two small prime numbers or two numbers that aren't RP with each other are chosen, then a valid e cannot 
    # be found sometimes; in these cases, the entire RSA scheme will not work.  Kill the process here.
    print('A valid e could not be found; use larger p and q values and/or make sure that p and q are prime. ' +\
          ' Process Terminated.')
    raise Exception('A valid e could not be found; use larger p and q values and/or make sure that p and q are prime') 

In [9]:
def Find_Private_Key_d(e, p, q):
    """
    Uses the Extended Euclidean Algorithm to calculate the private key, d, 
    for RSA decryption -- d is the modular inverse of e mod (p-1)(q-1).
    
    Args:
        e (int) - the e value from the user's public key
        p (int) - the first of two prime numbers for the calculation
        q (int) - the second prime number for the calculation
        
    Returns:
        (int) - the d value to be used in RSA decryption
    """
    pm1 = p - 1
    qm1 = q - 1
    p1q1 = pm1 * qm1
    return extended_euclidean_alg(e, p1q1)[1] # Corresponds to the Inverse

<section id='putting'>
#### 5. Puting things all together

In [10]:
def Encode(message, public_key):
    """
    RSA encryption of a message using a given public key with Fast Modular Exponentiation.
    
    Args:
        message (string) - The message to be encrypted
        public_key (tuple) - contains the e and n values to be used to encrypt the message 
                                
    Returns:
        (list) - the encrypted message character by character (in ascii representation) in a list
    """
    e, n = public_key
    e_bin = Convert_Binary_String(e)
    
    cipher_message = []
    ascii_message = Convert_Text(message) # Convert the message to ascii
    for char_num in ascii_message:
        cipher_num = FME(char_num, e_bin, n) # Fast Modular Exponentiation
        cipher_message.append(cipher_num)
    return cipher_message

In [11]:
def Decode(cipher, private_key, public_key):
    """
    RSA decryption of an encrypted message (a cipher) using given private and public keys with 
    Fast Modular Exponentiation.
    
    Args:
        cipher (list) - encrypted ascii characters from an original unicode message
        private_key (int) - the modular inverse of e mod (p-1)(q-1)
        public_key (tuple) - contains the e and n values to be used to encrypt the message 
        
    Returns:
        (string) - The decrypted message
        
    """
    decrypted_message = []
    
    d = private_key
    d_bin = Convert_Binary_String(d)
    
    n = public_key[1]
    
    for c in cipher:
        message_num = FME(c, d_bin, n)
        decrypted_message.append(message_num) # Fast Modular Exponentiation
    return Convert_Num(decrypted_message)

<section id='my_functions'>
#### 6. My Functions

<section id='prime_related'>
6.1. Prime-Related Functions

In [12]:
def primes_sieve(number):
    """
    Simulates the sieve of eratosthenes to generate all of the primes between 1 and an inputted number.  Works by 
    iterating all primes from 2 to the square root of the inputted number -- if a number is divisible by a prime, it
    itself is not prime.
    
    Args:
        number (int) - The largest number to consider
    
    Returns:
        (list) - A list of the prime numbers between 1 and an inputted number
    """
    assert isinstance(number, int) and number > 1
    prime_list = [i for i in range(2, number + 1)]
    
    i = 0
    max_prime_divisor = math.floor(math.sqrt(number)) # Only consider primes up to and including the 
                                                      # sqrt of number (Theorem 2)
    

    while prime_list[i] <= max_prime_divisor:
        for num in prime_list[(i+1):]: # Check all numbers larger than prime_list[i]
            if num % prime_list[i] == 0:
                prime_list.remove(num)
        i += 1
    return prime_list

def is_prime(number):
    """
    Determines whether or not an inputted number is prime or not.  Uses theorem 2:  Tests to see if a number is 
    divisible by any primes from 2 to square root of the number.
    
    Args:
        number (int) - The inputted number to test
    
    Returns:
        (Bool) - Whether the number is prime or not
    """
    assert isinstance(number, int) and number > 1
    
    if number == 2 or number == 3:
        # The primes_sieve function requires that numbers are greater than 3 because of the utilization of Theorem 2--
        # Their square roots make it difficult to construct a list of primes that can be used here.
        # Therefore, I am treating these numbers are base cases.
        return True
    
    max_prime_divisor = math.floor(math.sqrt(number)) # Only consider primes up to and including the
                                                      # sqrt of number (Theorem 2)
    prime_list = primes_sieve(max_prime_divisor)
    
    for prime in prime_list:
        if number % prime == 0:
            return False
        return True
    
def test_is_prime():
    """
    Tests the is_prime function -- should return True for all primes than 1000
    
    Args:
        None
    
    Returns:
        (Bool) - Whether or not the function works
    """
    primes_less_than_1000 = [2,   3,   5,   7,  11,  13,  17,  19,  23,  29,  31,  37,  41, 43,  47,  53,  59,  61,  67,  
     71,  73,  79,  83,  89,  97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167,
     173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313,
     317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467,
     479, 487, 491, 499, 503, 509, 521, 523, 541, 547, 557, 563, 569, 571, 577, 587, 593, 599, 601, 607, 613, 617, 619, 631, 641, 643,
     647, 653, 659, 661, 673, 677, 683, 691, 701, 709, 719, 727, 733, 739, 743, 751, 757, 761, 769, 773, 787, 797, 809, 811, 821, 823, 
     827, 829, 839, 853, 857, 859, 863, 877, 881, 883, 887, 907, 911, 919, 929, 937, 941, 947, 953, 967, 971, 977, 983, 991, 997 ] 
    
    for prime in primes_less_than_1000:
        if not is_prime(prime):
            return False
    return True

def load_primes(max_val=49979687):
    """
    Loads the first 3 milion prime numbers (from 2 to nearly 50 million) from an external text file that was obtained
    from https://primes.utm.edu/lists/small/millions/ as per the suggestion of Thomas Andrew Cochran.
    
    Args:
        max_val (int) - optional, used to abbreviate the list at the specified value -- the default value is the 
                        last number in the file
                        
    Returns:
        (list) - List of primes from 2 to whatever endpoint is specified (<= 49,979,687)
    """ 
    assert max_val <= 49979687, "Value must be less than 49,979,687"
    
    primes_list_processed = []
    with open('primes.txt', 'r') as the_primes:
        line = the_primes.readline()
        i = 1
        while line:
            # Because of the variable amounts of white-space between digits, filter all sections down to two spaces
            line_stripped = line.replace("        ", "  ").\
                                                        replace("       ", "  ").replace("      ", "  ").\
                                                        replace("     ", "  ").replace("    ", "  ").\
                                                        replace("   ", "  ")
            primes_in_line = (line_stripped.strip()).split("  ") # the primes are separated by two spaces 
                                                                 # in the text file
            primes_in_line = list(map(int, primes_in_line))
            primes_list_processed += primes_in_line # concatenate the next line to the final primes list
            
            last_number = primes_in_line[-1]
            if last_number > max_val:
                return primes_list_processed
            
            the_primes.readline() # Skip a blank line
            line = the_primes.readline()
            i += 1
        return primes_list_processes

<section id='key_related'>
6.2. Key-related functions

In [13]:
def pick_random_p_q(max_range=200, min_range=2):
    """
    Chooses two prime numbers in an inputted range
    
    Args:
        max_range (int) - The largest number to consider in the selection (200 is default)
        min_range (int) - The smallest number to consider in the selection (2 is default)
    
    Returns:
        (tuple) - index 1 is p, index 2 is q
    """
    assert isinstance(min_range, int) and isinstance(max_range, int)
    many_primes = load_primes(max_range)
    range_primes = [i for i in many_primes if i >= min_range]
    
    p = random.choice(range_primes)
    range_primes.remove(p)
    q = random.choice(range_primes)
    return p, q

def get_p_q():
    """
    Walks the user through selecting their rsa p and q values -- they can either choose two prime numbers themselves
    or have a pseudo-random selection the numbers
    
    Args:
        None
    
    Returns:
        (tuple) - index 1 is p, index 2 is q
    """
    while True:
        user_keys = (input('Do you want to:\n1) Choose your own keys\n2) Get pseudo-random keys\n(Select 1, 2)\n').strip())
        print()
        if user_keys in ['1', '2']:
            break
    
    if user_keys == '1': # User Selects their own p and q
        while True:
            try:
                user_p = int((input('Select p: ')).strip())
                user_q = int((input('Select q: ')).strip())
                assert (user_p != user_q) and (user_p > 0) and (user_q > 0) and is_prime(user_p) and is_prime(user_q)
            except:
                print("One or both of the keys are invalid, please enter positive prime numbers\n")
            else:
                return user_p, user_q
            
    else: # Pseudo-random generation of p and q
        while True:
            try:
                print('Choose the range of number to pick a prime number from.\n TIP: use larger ' +\
                      'numbers, as small primes may not yield a valid e for the RSA Encryption.\n')
                time.sleep(2) # Adds a delay so that the user can read the tip above
                user_range_min = int((input('Lowest Number to consider? (50 is the default)\n')).strip())
                user_range_max = int((input('Highest Number to consider? (200 is the default)\n')).strip())
                assert(user_range_max > user_range_min)
            except:
                print('Please try again.\n')
            else:
                print()
                break
                
    user_p, user_q = pick_random_p_q(user_range_max, user_range_min)
    print("You've selected p = {} and q = {}\n".format(user_p, user_q))
    return user_p, user_q

<section id='user_functionality'>
6.3. User Functionality

In [38]:
def user_generate_keys():
    """
    Conducts the user interface for getting/generating the RSA keys; walks through getting the p and q values, 
    then calculates the n, e, and d values to be used for both the public and private keys.  Prints all values
    revelant to the user's private and public key at the end.
    
    Args:
        None
        
    Return:
        None, only prints the relevant keys
    """
    rsa_p, rsa_q = get_p_q()
    rsa_n = rsa_p * rsa_q
    rsa_e = Find_Public_Key_e(rsa_p, rsa_q)
    rsa_d = Find_Private_Key_d(rsa_e, rsa_p, rsa_q)
    print('\nPublic Key (e,n): ({}, {})\nPrivate Key (d): {}\n'.format(rsa_e, rsa_n, rsa_d))
    print('You are advised to write down the values you receive from the program for safekeeping.\n')
    return 
    
def user_encode():
    """
    Conducts getting the user input for the user's public_key and message in order to encrypt that message. Prints
    the resulting cipher from the message.
    
    Args:
        None
        
    Returns:
        None, only prints the encrypted message (the cipher) as a comma-space-delinated sequence
    """
    while True:
        try:
            user_public_key = input('What is the public key that you wish to use? (must be "e, n" format) \n')
            rsa_public_key = tuple(map(int, user_public_key.split(', ')))
            print()
        except: 
            'Please try again following the "e, n" format; example: (5, 453727) would be entered as 5, 453727'
        else:
            break
    rsa_message = input('What is your message that you would you like to encrypt? \n')
    print()
    rsa_cipher = Encode(rsa_message, rsa_public_key)
    print('Encrypted Message:', ', '.join(map(str, rsa_cipher)))
    print('\nYou are advised to copy the encrypted message to your clipboard and save it elsewhere.\n')
    return 

def user_decode():
    """
    Conducts getting the user input for the user's public key, private key, and cipher in order to decrypt
    that cipher. Prints the resulting message from the cipher.
    
    Args:
        None
        
    Returns:
        None, only prints the decrypted cipher (the message)
    
    """
    # Public Key Prompt
    while True:
        try:
            user_public_key = input('What is the public key that you wish to use? (must be "e, n" format) \n')
            rsa_public_key = tuple(map(int, user_public_key.split(', ')))
            print()
        except: 
            'Please try again following the "e, n" format; example: (5, 453727) would be entered as 5, 453727'
        else: 
            break

    # Private Key Prompt     
    while True:
        try:
            rsa_private_key = int((input('What is the private key that you wish to use? \n')).strip())
            print()
        except: 
            'Please try again'
        else: 
            break

    # Encrypted Message
    while True:       
        user_cipher = input(('Enter the encrypted message (string of numbers separated by commas); ' +\
                            'example: 1234, 525, 235, 1111, .. \n')).strip()
        print()
        try:
            rsa_cipher = map(int, user_cipher.split(', '))
        except:
            print('Try again, be sure to use the format listed before (a string of numbers separated by commas)\n')
        else:
            break

    # Decrypt Message
    rsa_decrypted = Decode(rsa_cipher, rsa_private_key, rsa_public_key)
    print('Decrypted Message:', rsa_decrypted)
    print('\nYou are advised to copy the encrypted message to your clipboard and save it elsewhere.\n')
    return 

In [39]:
def main():
    """
    The main wrapper for the user-interface. Combines all of the user-functionality functions in a neat, logical
    sequence that is easier to follow.  
    
    Allows the user to:
        - Generate RSA keys (using their own primes or pseudorandomly-generated primes)
        - Encrypt Messages using a public key
        - Decrypt Messages using public and private keys
        
    For more information about the Functionality of any of these processes -- consult chapters 1 - 6
    
    Args:
        None
        
    Returns:
        None, only prints the corresponding results from the appropriate functions invoked
    """ 
    while True:
        user_select = input(('Do you want to: \n1) Generate Keys\n2) Encode a ' +
                             'Message\n3) Decode a Message\n(Select 1, 2, or 3)\n ')).strip()
        print()
        if user_select in ['1', '2', '3']:
            break
        print('Please try again.\n')
    
    # Generate Keys    
    if user_select == '1':
        user_generate_keys()
    
    # Encode a Message
    elif user_select == '2':
        user_encode()
    
    # Decode a Message
    else:
        user_decode();
        
    while True:
        user_continue = (input(('Do you want continue using the RSA interface? (y/n)\n'))).strip().lower()
        if user_continue in ['y', 'n']:
            break
        print('Please enter "y" or "n"')
        
    if user_continue == 'y':
        main() # recursively call the main function
    return

<section id='user_interface'>
#### 7. User Interface

In [48]:
if __name__ == '__main__':
    """
    The Grand Wrapper for the interface.  The outputs should logically guide the user through using the 
    interface to work with RSA; however, consult the main function defition above (6.3) for more information.
    
    - Generate RSA keys (using their own primes or pseudorandomly-generated primes)
    - Encrypt Messages using a public key
    - Decrypt Messages using public and private keys
    
    """
    main() 

Do you want to: 
1) Generate Keys
2) Encode a Message
3) Decode a Message
(Select 1, 2, or 3)
 1

Do you want to:
1) Choose your own keys
2) Get pseudo-random keys
(Select 1, 2)
1

Select p: 11171
Select q: 11159

Public Key (e,n): (3, 124657189)
Private Key (d): 83089907

You are advised to write down the values you receive from the program for safekeeping.

Do you want continue using the RSA interface? (y/n)
n


### Demonstate how your RSA works below, by encoding and decoding a sample message here using your main above and sample inputs. Describe how this works and why you choose to put it together the way you did.


<section id='section_4'>
#### 8. "Section 4" Demo/Explanation
    
I actually opted to use a line-by-line user interface for generating values relevant to RSA.  In the main function, the user is prompted to either 1) Generate Keys, 2) Encode a Message, or 3) Decode a message. Users are advised to write down/copy the values they receive. After a user goes through the steps of one of the pathways (1, 2, or 3), they will be prompted if they wish to continue using the program. There is also extensive error-handling to ensure that the user is using the RSA schema correctly.

I chose this method, because it abstracts a lot of the mathematical/programming understanding away from the user (allowing them to focus on just using RSA as an application. All the user needs to do is run the block in the "User Interface" section (or run the main function straight up).  This also allows for a fast/convenient way for a user to use RSA.  It should be noted that this approach does not come without disadvantage, as scaling this program would take some effort on the backend (even with a command line macro/script).

Below is an example of how the program works option by option.

<b> Option 1, Generate Keys:</b>
If the user selects 'Generate Keys', the user is then prompted to either enter two prime numbers (must be prime or the code will not continue to the key calculations) or have two prime numbers generated for them (using a pseudorandom selection scheme). The user will then receive their public and private keys based on those prime numbers.

Example Inputs/Outputs (Psuedorandomly generated primes between 10000 and 20000):

![image.png](option_1.png)

Example Inputs/Outputs (User Selected):

![image.png](option_1b.png)

<b> Option 2, Encoding a Message: </b>
Then, using those public and private keys, the user can run the the second option (encode a message).  They will be prompted for their public key and a message that they would like to encrypt.  After entering those values correctly, the program will print out a comma-delinated sequence of ascii characters (corresponding to the encrypted message).

Example Inputs/Outputs:

![image.png](option_2.png)

<b> Option 3, Decoding a Message: </b>
The final functionality is decryption, the user can use their private and public keys to decrypt an encrypted message (so as long as the message was encrypted using their public key). Like in the encoding option, the user is prompted to enter their public key in "e, n" format. Next, they are prompted for their private key, d. And finally, they are prompted for the message they would like to decrypt (as a sequence of comma-delinated values -- the format that a cipher is printed as from option 2). The result is printed to the screen.

Example Inputs/Outputs:

![image.png](option_3.png)

<section id='section_5'>
#### 9. Section "5" Narrative

I have been exchanging keys and messages mostly with people who were able to finish coding the test, so there have only been a few people to correspond with (Thomas, Regean, Beth, and Brian namely).  At first, I was just generating my own keys and encrypting and decrypting my own messages/ciphers, which allowed me to build up my codebase.  Then, I took my classmates public and private keys from Piazza to decrypt some messages (this was a good proof of concept that everything was working, even outside the bubble that was my own codebase).  Next, I wanted to encrypt some messages for my classmates using public keys that they posted.  Me being me, I always like to have some fun with these types of exchanges so I encrypted and sent youtube video urls to them.  It was instant gratification to have them send me relevant youtube video urls back (using the public key that I posted on Piazza).

When it came time to decrypt some of the public keys and ciphers that did not include the public keys; however, I quickly found that my brute force factoring algorithm was not nearly as efficient as I thought it was (I really thought that I had engineered that thing to handle the n's for this class).  In fact, I was really only able to crack very small n values, and was forced to stare at my screen for up a half hour before my program would just crash trying to factor larger values of n (these were Brian's public keys, as you'll see below in <a href='#code_cracking'>Chapter 10.3</a>.  I was so confused to see that other were able to crack his code -- Thomas told me that I should include a list of prime numbers from an external text file to save time with the prime list generation (at the time, I was using an implementation the Sieve of Eratosthenes -- which turned out to be wildly inefficient for large numbers).  After taking this advance, my brute force factorer fared slightly better, but I was still not able to crack Brian codes.

Finally, I was directed to Beth's post on using the Pollard's Rho algorithm for more efficient factoring.  Desperate to crack the code, I started reading into the theory behind the algorithm.  I actually only took some of the parts of the code originally, since I thought the "pseduo-random' function could be any pseudorandom function, so I just used the random module to pick values of a and b (between 2 and n).  Finally!  After ~100,000,000 iterations, I had finally cracked Brian's code (this was the highlight of my excitement with the project -- it reminded me of mining cryptocurrencies; waiting, waiting, waiting, reward!  Very similar to gambling psychology, I suppose).  What was cool was that each time I wanted to crack the same code, it would take a different number of attempts: sometimes 100 million sometimes just 2 million.  It was fun to watch it work.

However, I realized that Brian had said that his Pollard's Rho was cracking everyone's codes in less than a second (which wasn't obviously the case for me).  In order to reach Brian's efficiency, I went back to Beth's code, and implemented the pseudorandom function f(x) = x^2 + 2 mod n.  Immediately I saw the results:  the codes that were taking (sometimes) over 100 million attempts were being calculated in 3000 attempts, everytime (no more variation).  Even Brian's very large n value (the last cipher in <a href='#code_cracking'>Chapter 10.3</a> was being cracked in less than 100,000 attempts.  It was very satisfying to see the progression.

Honestly, the project was fairly straight-forward to me conceptually.  Although, I should note that I did not use any version control for this project, so whenever I needed to make a change that was suggested by my classmates, I was working without a real backup -- a little stressful at times.  Most of my challenges, though, came with the design of the final interface (making sure that all of the formatting was correct and that I could account for as many user errors as possible) and also the design of overall Jupyter file.  I wanted it to read through easily, be as modular as possible, and have a logical structure all around.  Regardless, I still want to thank Reagan, Thomas, Brian, and Beth for the ideas in improving the performance of my program.

<section id='section_6'>
#### 10. Section #6 Factoring and Code Breaking

<section id='naive'>
10.1. Naive, Brute Force Factoring

In [49]:
def n_breaker(n):
    """
    Brute Force Factorer.  Utilizes the second theorem to cut down on the number values that need to be tested.  Works 
    well for smaller n's, but gets very inefficient for larger values of n.
    
    Args:
        n (int) - The number to be factored
    
    Returns:
        (tuple) - index 1: First divisor, index 2: Second Divisor
    """   
    max_prime_divisor = math.floor(math.sqrt(n)) # Only consider primes up to and including the 
                                                      # sqrt of number (Theorem 2)
        
    primes_list = load_primes(max_val=max_prime_divisor)
    
    for prime in primes_list:
        if n % prime == 0:
            divisor1 = prime # divisor found!
            break
    
    divisor2 = int(n / divisor1)
    return divisor1, divisor2

def naive_code_breaker(public_key, cipher):
    """
    Wrapper for the n_breaker.  What the user would use to try to factor a number with the n_breaker function.
    
    Args:
        public_key (tuple) - user's public key, index 1: e, index 2: n
        cipher (list) - list of encypted ascii values
        
    Returns:
        (string) - the decrypted message
    """
    e, n = public_key
    
    p, q = n_breaker(n)
    print("p = {}, q = {}".format(p, q))
    d = Find_Private_Key_d(e, p, q)
    
    return Decode(cipher, d, public_key)

<section id='pollards'>
10.2. Advanced Factoring with Pollard's Rho

In [107]:
def pollards_rand_num(x, n):
    """
    The "random" function (f(x) = x^2 + 2 mod n) that generates the divisors to test in the pollard's rho algorithm
    
    Args:
        x (int) - input number
        n (int) - number to be factored
        
    Returns:
        (int) - Generated from the function 
    """
    return (x ** 2 + 2) % n

def pollards_rho(n, attempts=100000000):
    """
    Utilizes the Pollard's Rho Algorithm for factoring a number. Due to the probablistic nature of the algorithm,
    the runtime for finding a factor could be ms or minutes. The implementation was adapted from Beth Spade's coding of 
    it in C (http://www.cs.colorado.edu/~srirams/courses/csci2824-spr14/pollardsRho.html)
    
    Terminates after 100,000,000 attempts (default) or a user-specfied amount of attempts to factor the number.
    If this is the case, the number is either tremendously large (in which case, increase attempts) or
    the pseudorandom function reaches an infite loop state.
    
    
    Args:
        n = number to be factored
        attempts = maximum nubmer of attempts to find a factor, code terminates is this number is elapsed
        
    Returns:
        None - if no factor is found or the number is found to be prime, "Failed" is returned
        (tuple) - the primes/divisors that make up the number
    """
    
    max_divisor = math.floor(math.sqrt(n))
    
    i = 1 # attempt counter
    a, b = 2, 2
    
    while True:
        a = pollards_rand_num(a, n)
        b = pollards_rand_num(pollards_rand_num(b, n), n)
    
        p = math.gcd(abs(a-b), n)
        
        if p > 1:
            print('\n{} number of attempts to crack n\n'.format(i))
            q = int(n / p)
            if q == 1:
                print("Prime Number")
                return
            return q, p
        
        i += 1
        if i == attempts:
            print("Failed")
            return

def pollards_rho_code_breaker(public_key, cipher):
    """
    Wrapper for the pollards_rho algorithm to make the sequence more logical for the user. Useful for factoring 
    larger n values, due to its probabilistic nature.
    
    Args:
        public_key (tuple) - user public key, index 1: e, index 2: n (the number to be factored)
        cipher (list) - list of encrypted ascii values to be decrypted
    """
    e, n = public_key
    try:
        p, q = pollards_rho(n)
        print("p = {}, q = {}".format(p, q))
    except:
        return "Failed to break the code."
    
    d = Find_Private_Key_d(e, p, q)
    
    return Decode(cipher, d, public_key)

<section id='code_cracking'>
10.3. Code Cracking!

In [101]:
# Crack my own cipher
my_cipher = [293246, 393746, 393746, 206425, 307692, 267526, 212872, 212872, 218761, 218761, 218761, 424645, 120646,
               68225, 391717, 393746, 391717, 58674, 422000, 424645, 236306, 68225, 356979, 212872, 218761, 103055,
               393746, 236306, 293246, 135594, 208501, 210354, 357593, 192731, 362010, 335824, 103055, 314963, 20431,
               434533, 257055, 110686, 41260, 286670, 382569, 60135, 310647, 422000, 357593, 210354, 257055, 286670, 
               239327, 382569, 307692, 393746, 210354, 437333, 98900, 287282, 96822, 357593, 310647, 218761, 236556,
               18112, 257055, 357593, 393746, 225704, 208501, 263121, 240579, 96822, 46056, 117665, 68225, 239327,
               313231, 50055, 402844, 268941, 357593, 117665, 268941, 240579, 448842, 41260, 402844, 281594, 357593]
print("My Message: " + naive_code_breaker((5, 453727), my_cipher))

p = 619, q = 733
My Message: https://www.youtube.com/watch?v=x3k2a-X41AY&index=1&list=PLCZxdwSW1xtHv0rZBTolq9_fxTfrRY_Jx


In [102]:
# Crack Gregory GIORDANO's 
greg_cipher = [30230, 34467, 35867, 13022, 23607, 29617, 35867, 31944, 34467, 37164, 20354, 28406, 35867, 
               16953, 35867, 11419, 36622, 21806, 11419, 36759, 17296, 35867, 14147, 34467, 6592, 35867, 
               17296, 34467, 35867, 20354, 4554, 11419, 10799]
greg_pub_key = (5, 39203)
print("Greg's Mesage: " + naive_code_breaker(greg_pub_key, greg_cipher))

p = 197, q = 199
Greg's Mesage: No Mr. Bond, I expect you to die!


In [108]:
# Crack Brian Solar's
brian_cipher = [6060711605323, 207616015289871, 194871710000000, 266001988046875, 140710042265625, 100000000000000, 107213535210701, 250226879128704, 80798284478113, 182803912081669, 207616015289871, 266001988046875, 34359738368, 107213535210701, 266001988046875, 282621973446656, 80798284478113, 266001988046875, 34359738368, 318547390056832, 107213535210701, 250226879128704, 100000000000000, 80798284478113, 100000000000000, 107213535210701, 266001988046875, 34359738368, 93206534790699, 207616015289871, 182803912081669, 207616015289871, 34359738368, 107213535210701, 318547390056832, 140710042265625, 100000000000000, 107213535210701, 194871710000000, 282621973446656, 107213535210701, 266001988046875, 34359738368, 221068140740608, 207616015289871, 250226879128704, 34359738368, 266001988046875, 140710042265625, 34359738368, 182803912081669, 107213535210701, 266001988046875, 182803912081669, 80798284478113, 266001988046875, 319277809664, 34359738368, 235260548044817, 300124211606973, 107213535210701, 34359738368, 282621973446656, 207616015289871, 100000000000000, 207616015289871, 266001988046875, 34359738368, 207616015289871, 266001988046875, 34359738368, 131593177923584, 207616015289871, 182803912081669, 107213535210701, 194871710000000, 266001988046875, 34359738368, 266001988046875, 515031406547273, 207616015289871, 34359738368, 93206534790699, 250226879128704, 140710042265625, 80798284478113, 100000000000000, 207616015289871, 266001988046875, 34359738368, 140710042265625, 122987386542487, 300124211606973, 80798284478113, 140710042265625, 266001988046875, 319277809664, 34359738368, 100000000000000, 207616015289871, 282621973446656, 80798284478113, 100000000000000, 207616015289871, 266001988046875, 34359738368, 221068140740608, 107213535210701, 171382426877952, 207616015289871, 34359738368, 6060711605323, 250226879128704, 140710042265625, 80798284478113, 100000000000000, 207616015289871, 250226879128704, 34359738368, 100000000000000, 107213535210701, 34359738368, 93206534790699, 107213535210701, 250226879128704, 282621973446656, 207616015289871, 266001988046875, 34359738368, 100000000000000, 140710042265625, 250226879128704, 107213535210701, 140710042265625, 282621973446656, 207616015289871, 266001988046875, 34359738368, 140710042265625, 194871710000000, 80798284478113, 171382426877952, 140710042265625, 107213535210701, 194871710000000, 685653754644797, 318547390056832, 107213535210701, 140710042265625, 266001988046875, 319277809664, 34359738368, 235260548044817, 300124211606973, 107213535210701, 34359738368, 107213535210701, 194871710000000, 282621973446656, 250226879128704, 107213535210701, 34359738368, 107213535210701, 266001988046875, 282621973446656, 107213535210701, 266001988046875, 34359738368, 107213535210701, 266001988046875, 282621973446656, 515031406547273, 207616015289871, 34359738368, 80798284478113, 34359738368, 318547390056832, 140710042265625, 100000000000000, 80798284478113, 319277809664, 34359738368, 80798284478113, 34359738368, 171382426877952, 140710042265625, 86812553324672, 107213535210701, 250226879128704, 100000000000000, 80798284478113, 100000000000000, 107213535210701, 34359738368, 107213535210701, 34359738368, 80798284478113, 34359738368, 221068140740608, 250226879128704, 207616015289871, 93206534790699, 300124211606973, 250226879128704, 80798284478113, 34359738368, 100000000000000, 80798284478113, 34359738368, 114868566764928, 107213535210701, 171382426877952, 140710042265625, 93206534790699, 140710042265625, 100000000000000, 80798284478113, 100000000000000, 107213535210701]
brian_pub_key = (7, 1018116866812351)

# My Naive Brute Force approach takes too long for my little Brute Forcer and kills the kernel, so resorting to 
# Pollards Rho
print("Brian Message: " + pollards_rho_code_breaker(brian_pub_key, brian_cipher))


3024 number of attempts to crack n

p = 76425211, q = 13321741
Brian Message: Consideramos estas verdades como evidentes por si mesmas, que todos os homens são criados iguais, dotados pelo Criador de certos direitos inalienáveis, que entre estes estão a vida, a liberdade e a procura da felicidade


In [109]:
# Crack Brian Solar's Second Cipher
brian_cipher2 = [10030613004288, 107213535210701, 171382426877952, 171382426877952, 207616015289871, 34359738368, 6722988818432, 80798284478113, 250226879128704, 160578147647843, 194871710000000, 107213535210701, 266001988046875, 266001988046875, 34359738368, 16048523266853, 379749833583241, 34359738368, 19203908986159, 171382426877952, 100000000000000, 34359738368, 8235430000000, 250226879128704, 140710042265625, 107213535210701, 194871710000000, 100000000000000]
brian_pub_key2 = (7, 1671976569351094677533)

# My Naive Brute Force approach takes too long for my little Brute Forcer and kills the kernel, so resorting to 
# Pollards Rho
print("Brian Message: " + pollards_rho_code_breaker(brian_pub_key2, brian_cipher2))


81701 number of attempts to crack n

p = 46974051941, q = 35593620313
Brian Message: Hello Darkness My Old Friend


<section id='section_7'>
#### 11. Section "7" Code Breaking Description

It is very interesting to compare the types of n-factoring algorithms that I have implemented.  For example, my brute force factorer (n_breaker) does just fine with cracking smaller n values (less than a second runtime); however, when I tried to factor larger n values, it would just run and run and run (I let it try to factor Brian Solar's n for 25 minutes before terminating it) even though I have tried to optimize it by 1) using theorem two to eliminate many numbers to test and 2) loading in a list of prime numbers from a text file in the directory.  It just isn't feasible for these n's (which aren't even close to size that real RSA encryption use).  For my Pollard's Rho algorithm, however, the calculation is extremely fast -- less than a second for even large n's.  If I were going to try to implement my own RSA schema, I'd probably use this algorithm as a starting point to figure out how big the n values ought to be.

<section id='section_8'>
#### 12. Section "8" Code Breaking Successes

I ended up breaking several codes (listed above in <a href='#code_cracking'>chapter 10.3</a>).  I've included a cipher that I created using my program and also three ciphers that were posted by my classmates on Piazza.  Notice, that I used my naive brute force code cracker for the first two (which were much smaller n values), but I had to resort to the advanced factoring algorithm (Pollard's Rho implementation) for the final ciphers made by Brian (his n values were quite large, but the Pollard's Rho took care of them very quickly).

<section id='section_9'>
#### 13. Section "9" Theorem 2 Importance

Theorem 2 allows us to cut down the numbers tests that need to be performed tremendously.  The theorem states that any composite number must have a prime factor less than or equal to the square root of that number.  This means that instead of testing every number from 2 to n-1 (or every odd number greater than 2), we only need to test the prime numbers less than or equal to the square root of the number.  If a number is prime, then none of the primes from 2 to square root of that number will be a divisor.

For example, consider the prime number 101.  It's square root is slightly larger than 10, so the largest number to consider is 10.  Now find all the prime numbers from 2 to 10, which are {2, 3, 5, 7}.  None of these numbers divide 101; therefore, 101 is a prime number.  So, we only had to test 4 numbers, whereas if we were using the brute force factoring scheme testing 2 to n-1, we would have had to have tested 99 numbers.  The runtime savings of using theorem 2 in a factoring algorithm just keep increasing as the number keeps growing.

<section id='section_10'>
#### 14. Section "10" Conclusion/Answering Questions: 

My Previous Questions (just picking three):<br>
<i>1. Why do the prime numbers chosen as the prime factors of n have to be of “roughly equal size”?</i><br>
I think this makes it harder for someone to crack the n, because if you have one very large factor, then you are left with one smaller factor; therefore, it opens up the door find that smaller factor (or a multiple of it, if using an algorithm like Pollard's Rho).

<i>2. When public/private keys are generated, what is doing the value selections?  A pseudo-random number generator?</i><br>
A program most likely uses a pseudo-random prime selector (out of a pool of very large prime numbers), since it would be highly impractical to have the user find two extremely large prime numbers everytime they wanted to generate a public/private key pair.

<i>3. I’ve used PGP encryption before, and it seems to use a similar pattern of public-key exchange with a private key decryption.  Are these encryption methods related at all?</i><br>
Well, we didn't cover PGP in this course; however, I now realize that PGP is not an algorithm that does the encryption/decryption, but actually the software that can use many different types of cryptographic algorithms (RSA included!) This definitely makes more sense now that I have actually worked through an implementation of an RSA scheme; next step -- rebuild PGP!

I've learned a lot more than just the topics covered by these questions, but the biggest takeaway for me is the amount of computer power needed to work with big number theory concepts.  I don't think I really understood that finite computations could really take all that long to do -- for example, I thought after optimizing my brute force factorer that it would be able to crank through just about anything; however, it was miserably slow for those large prime numbers.  But that's the thing, the numbers that I was trying to crack weren't even all that large compared to the numbers used in real-RSA implementations!  It makes me realize why studying computer science is so important (compared to just learning how to program in a particular language).  Speed matters... A lot...